## Thuật toán Newton lùi cho phương pháp giải đa thức nội suy 

In [2]:
import numpy as np
import pandas as pd
import math
from numpy.polynomial import polynomial as P

def load_horizontal_data(filepath):
    """
    Hàm đọc dữ liệu dạng hàng ngang từ file txt một cách mạnh mẽ.
    """
    try:
        with open(filepath, 'r', encoding='utf-8-sig') as f:
            line = f.readline()
        if not line: return np.array([])
        line = line.strip().replace(',', '.')
        data = np.fromstring(line, sep=' ')
        return data
    except FileNotFoundError:
        raise IOError(f"LỖI: Không tìm thấy file '{filepath}'.")
    except Exception as e:
        raise IOError(f"LỖI: Không thể đọc file '{filepath}'. Lỗi: {e}")

# --- Chương trình chính cho Ô nạp dữ liệu ---
try:
    # 1. Nạp dữ liệu thô
    x_coords_raw = load_horizontal_data('FileX.txt')
    y_coords_raw = load_horizontal_data('FileY.txt')

    if len(x_coords_raw) == 0 or len(y_coords_raw) == 0:
        raise ValueError("LỖI: Một trong hai file không chứa dữ liệu số hợp lệ.")
    if len(x_coords_raw) != len(y_coords_raw):
        raise ValueError("LỖI: Số lượng điểm x và y không khớp.")
    if len(np.unique(x_coords_raw)) != len(x_coords_raw):
        raise ValueError("LỖI: Các giá trị x bị trùng lặp.")

    # 2. SẮP XẾP DỮ LIỆU GIẢM DẦN (THEO YÊU CẦU NEWTON LÙI)
    print("Đang sắp xếp dữ liệu theo x giảm dần...")
    # Lấy chỉ số sắp xếp giảm dần
    sort_indices_descending = np.argsort(x_coords_raw)[::-1]
    
    # Áp dụng chỉ số sắp xếp cho cả x và y
    x_coords = x_coords_raw[sort_indices_descending]
    y_coords = y_coords_raw[sort_indices_descending]

    # 3. In kết quả đã sắp xếp
    print("Dữ liệu đã được đọc và sắp xếp thành công!")
    print(f"Tổng số {len(x_coords)} điểm đã được nạp.")
    print("-" * 50)
    print("Các mốc x (đã sắp xếp giảm dần):", x_coords.tolist())
    print("Các giá trị y (tương ứng):", y_coords.tolist())
    
except (IOError, ValueError) as e:
    print(str(e))

Đang sắp xếp dữ liệu theo x giảm dần...
Dữ liệu đã được đọc và sắp xếp thành công!
Tổng số 7 điểm đã được nạp.
--------------------------------------------------
Các mốc x (đã sắp xếp giảm dần): [2.3, 2.1, 2.0, 1.8, 1.5, 1.3, 1.2]
Các giá trị y (tương ứng): [1.0378, 0.5597, 0.3464, 0.0032, -0.24, -0.2069, -0.1374]


In [3]:
# Giả định các biến x_coords và y_coords đã tồn tại từ các ô trước.

def create_lower_div_diff_table(x_values, y_values):
    """
    Tạo bảng tỷ sai phân dạng tam giác dưới.
    Các hệ số Newton c_k = f[x_0, ..., x_k] sẽ nằm trên đường chéo: table[k, k].
    """
    n = len(x_values)
    
    # 1. Tạo ma trận full bằng NaN
    table = np.full((n, n), np.nan)
    
    # 2. Cột đầu tiên (j=0) là các giá trị y
    table[:, 0] = y_values
    
    # 3. Tính toán các giá trị tỷ sai phân
    for j in range(1, n): # Cột (bậc tỷ sai phân)
        for i in range(j, n): # Hàng (bắt đầu từ đường chéo)
            # Công thức: f[x_{i-j}, ..., x_i] = (f[x_{i-j+1}, ..., x_i] - f[x_{i-j}, ..., x_{i-1}]) / (x_i - x_{i-j})
            numerator = table[i, j-1] - table[i-1, j-1]
            denominator = x_values[i] - x_values[i-j]
            table[i, j] = numerator / denominator
            
    return table

# --- Thực thi hàm ---
try:
    # 1. Tính toán bảng
    div_diff_table_lower = create_lower_div_diff_table(x_coords, y_coords)
    
    # 2. Tạo DataFrame để hiển thị
    headers = [f'y'] + [f'TSP{i}' for i in range(1, len(x_coords))]
    df_display = pd.DataFrame(div_diff_table_lower, columns=headers)
    df_display.insert(0, 'x', x_coords) # Thêm cột x vào đầu

    # 3. Hàm tô màu đường chéo
    def highlight_diagonal(data):
        style = 'background-color: yellow; font-weight: bold; color: black;'
        style_df = pd.DataFrame('', index=data.index, columns=data.columns)
        
        # Lặp và tô màu đường chéo
        # Bỏ qua cột 'x' (cột 0), bắt đầu từ cột 'y' (cột 1)
        for i in range(len(data.index)):
            col_idx_in_df = i + 1 # Cột 'y' là 1, 'TSP1' là 2, ...
            if col_idx_in_df < len(data.columns):
                val = data.iloc[i, col_idx_in_df]
                if not pd.isna(val): # Chỉ tô màu ô có giá trị
                    style_df.iloc[i, col_idx_in_df] = style
        return style_df

    # 4. Áp dụng style và định dạng
    print("--- Bảng Tỷ sai phân (Dạng tam giác dưới, tô màu đường chéo) ---")
    numeric_cols = df_display.select_dtypes(include=np.number).columns
    styled_df = df_display.style.apply(highlight_diagonal, axis=None).format(
        "{:g}", 
        subset=numeric_cols, 
        na_rep="" # Hiển thị ô trống thay vì 'NaN'
    )
    
    display(styled_df)
    
    # 5. In ra các hệ số Newton (chính là đường chéo của bảng tính)
    newton_coefficients = np.diag(div_diff_table_lower)
    print("\n--- Các hệ số Newton (giá trị trên đường chéo) ---")
    print(newton_coefficients)

except NameError:
    print("LỖI: Các biến 'x_coords' và 'y_coords' chưa được định nghĩa. Vui lòng chạy ô nạp dữ liệu trước.")

--- Bảng Tỷ sai phân (Dạng tam giác dưới, tô màu đường chéo) ---


,x,y,TSP1,TSP2,TSP3,TSP4,TSP5,TSP6
0,2.3,1.0378,,,,,,
1,2.1,0.5597,2.3905,,,,,
2,2,0.3464,2.133,0.858333,,,,
3,1.8,0.0032,1.716,1.39,-1.06333,,,
4,1.5,-0.24,0.810667,1.81067,-0.701111,-0.452778,,
5,1.3,-0.2069,-0.1655,1.95233,-0.202381,-0.623413,0.170635,
6,1.2,-0.1374,-0.695,1.765,0.312222,-0.643254,0.0220459,0.135081



--- Các hệ số Newton (giá trị trên đường chéo) ---
[ 1.0378      2.3905      0.85833333 -1.06333333 -0.45277778  0.17063492
  0.13508097]


In [4]:
# Giả định biến 'x_coords' đã được định nghĩa ở ô trên.

try:
    n = len(x_coords)
    
    # --- B1: Lấy các giá trị 'c' (giữ nguyên) ---
    # Lấy n-1 giá trị x đầu tiên.
    c_values = x_coords[:-1]
    
    # --- B3: Thiết lập số cột (ĐÃ THAY ĐỔI) ---
    # >>> SỬA LỖI Ở ĐÂY: Số cột bây giờ là n <<<
    num_cols = n 
    
    # Số hàng cần tính toán vẫn là n-1
    num_rows_to_calc = n - 1 

    # --- Bước khởi tạo ---
    product_table_rows = []
    
    # Hàng đầu tiên [0, 0, ..., 1] bây giờ sẽ có n phần tử
    initial_row = np.zeros(num_cols)
    initial_row[-1] = 1.0
    product_table_rows.append(initial_row)

    print("--- Bắt đầu xây dựng Bảng Tích ---")
    print(f"Số mốc nội suy n = {n}")
    print(f"Số cột trong bảng tính = {num_cols}") # Sẽ in ra 7 nếu n=7
    
    # --- B2: Lặp và tính toán các hàng (giữ nguyên) ---
    for i in range(num_rows_to_calc):
        c = c_values[i]
        a = product_table_rows[-1]
        b = np.zeros(num_cols) # Sẽ tạo hàng mới có n phần tử

        # Áp dụng công thức: b_k = a_{k+1} - a_k * c
        for k in range(num_cols):
            a_k_plus_1 = a[k+1] if k < num_cols - 1 else 0.0 
            a_k = a[k]
            b[k] = a_k_plus_1 - (a_k * c)
            
        product_table_rows.append(b)

    # --- Hiển thị kết quả ---
    product_table_data = np.array(product_table_rows)
    df_product = pd.DataFrame(product_table_data)
    
    c_column_display = [np.nan] + c_values.tolist()
    df_product.insert(0, 'Giá trị c (x)', c_column_display)

    print("\n--- Bảng Tích Hoàn Chỉnh ---")
    display(df_product.style.format("{:g}", na_rep=""))

except NameError:
    print("LỖI: Biến 'x_coords' chưa được định nghĩa. Vui lòng chạy ô nạp dữ liệu trước.")

--- Bắt đầu xây dựng Bảng Tích ---
Số mốc nội suy n = 7
Số cột trong bảng tính = 7

--- Bảng Tích Hoàn Chỉnh ---


,Giá trị c (x),0,1,2,3,4,5,6
0,,0,0,0,0,0,0,1
1,2.3,0,0,0,0,0,1,-2.3
2,2.1,0,0,0,0,1,-4.4,4.83
3,2,0,0,0,1,-6.4,13.63,-9.66
4,1.8,0,0,1,-8.2,25.15,-34.194,17.388
5,1.5,0,1,-9.7,37.45,-71.919,68.679,-26.082
6,1.3,1,-11,50.06,-120.604,162.174,-115.365,33.9066


In [5]:
# Giả định các biến 'newton_coefficients' và 'product_table_data' đã tồn tại.

# (Hàm phụ trợ để in đa thức - Đảm bảo đã chạy ở ô trước)
try:
    _ = format_polynomial_descending
except NameError:
    print("Đang định nghĩa hàm 'format_polynomial_descending'...")
    def format_polynomial_descending(coeffs_asc):
        # ... (code hàm format) ...
        coeffs_desc = coeffs_asc[::-1]; poly_str = ""; n = len(coeffs_desc) - 1
        for i, coeff in enumerate(coeffs_desc):
            if np.isclose(coeff, 0): continue
            power = n - i; sign = " - " if coeff < 0 else " + "; coeff_abs = abs(coeff)
            if poly_str == "": sign = "" if coeff > 0 else "-"
            if np.isclose(coeff_abs, 1) and power != 0: coeff_str = ""
            else: coeff_str = f"{coeff_abs:g}"
            if power == 1: var_str = "x"
            elif power == 0: var_str = ""
            else: var_str = f"x^{power}"
            separator = "*" if coeff_str != "" and var_str != "" else ""
            poly_str += f" {sign} {coeff_str}{separator}{var_str}"
        return poly_str.strip() if poly_str else "0"

# --- Bước 1: In Vector tham số Newton ---
print("--- B1: Vector tham số Newton (từ đường chéo Bảng Tỷ sai phân) ---")
try:
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(newton_coefficients)
except NameError:
    print("LỖI: Biến 'newton_coefficients' chưa được định nghĩa.")
    raise 
print("\n--- B1: Bảng Tích (Ma trận nhân) ---")
try:
    with np.printoptions(threshold=np.inf, precision=15, suppress=True, linewidth=np.inf):
        print(product_table_data)
except NameError:
    print("LỖI: Biến 'product_table_data' chưa được định nghĩa.")
    raise 
print("-" * 50)

# --- Bước 2: Thực hiện phép nhân ma trận ---
print("--- B2: Thực hiện phép nhân ---")
print("Vector_Newton @ Bảng_Tích")

try:
    # >>> THAY ĐỔI Ở ĐÂY: Đổi tên biến cho đúng <<<
    # Phép nhân này cho ra hệ số bậc GIẢM DẦN (An, A(n-1), ..., A0)
    final_coeffs_descending = newton_coefficients @ product_table_data
    
    # --- B3: In kết quả và đa thức ---
    print("\n--- Kết quả Vector hệ số cuối cùng (Hệ số P(x) bậc GIẢM DẦN) ---")
    with np.printoptions(threshold=np.inf, precision=15, suppress=True):
        print(final_coeffs_descending)
        
    print("\n--- B3: Đa thức nội suy P(x) hoàn chỉnh (từ kết quả nhân) ---")
    # Hàm 'format_polynomial_descending' cần hệ số bậc TĂNG DẦN (A0, ..., An)
    # Vì vậy, chúng ta đảo ngược (final_coeffs_descending[::-1]) trước khi gọi hàm
    print(f"P(x) =", format_polynomial_descending(final_coeffs_descending[::-1]))

except ValueError as e:
    print(f"\nLỖI KHI NHÂN MA TRẬN: {e}")
    print(f"Kích thước Vector Newton: {newton_coefficients.shape}")
    print(f"Kích thước Bảng Tích: {product_table_data.shape}")
except NameError as e:
     print(f"LỖI: Một trong các biến ('newton_coefficients' hoặc 'product_table_data') chưa được định nghĩa.")

print("=" * 50)
print("--- KẾT THÚC ---")

--- B1: Vector tham số Newton (từ đường chéo Bảng Tỷ sai phân) ---
[ 1.0378             2.390500000000003  0.858333333333353
 -1.063333333333277 -0.452777777777687  0.170634920635035
  0.135080968414427]

--- B1: Bảng Tích (Ma trận nhân) ---
[[   0.                   0.                   0.                   0.                   0.                   0.                   1.               ]
 [   0.                   0.                   0.                   0.                   0.                   1.                  -2.3              ]
 [   0.                   0.                   0.                   0.                   1.                  -4.4                  4.83             ]
 [   0.                   0.                   0.                   1.                  -6.4                 13.63                -9.66             ]
 [   0.                   0.                   1.                  -8.200000000000001   25.150000000000002  -34.194               17.388            ]
 [   0. 

## Thuật toán tính giá trị đạo hàm P(x') tại x = x'

In [7]:
# Giả định biến 'final_coeffs_ascending' đã được định nghĩa ở ô trên.
# Đây là vector hệ số P(x) theo bậc tăng dần (a_0, a_1, ..., a_n).

# --- 1. Thiết lập đầu vào ---
x_prime_input = 1.5                                             # Bạn có thể thay đổi giá trị x' ở đây

# --- 2. Hàm thuật toán Horner lặp ---
def tinh_P_va_cac_dao_ham(coeffs_asc, x_prime):
    """
    Tính giá trị của P(x') và tất cả các đạo hàm của nó P'(x'), P''(x'),...
    bằng phương pháp Horner lặp.

    Args:
        coeffs_asc (list or np.array): Các hệ số của P(x) theo bậc TĂNG DẦN.
        x_prime (float): Điểm x' cần tính giá trị.
    """
    
    # Horner lặp hoạt động với hệ số bậc giảm dần
    coeffs_desc = coeffs_asc[::-1]
    
    print("\n" + "="*50)
    print(f"Đa thức P(x) có hệ số (bậc giảm dần): {coeffs_desc.tolist()}")
    print(f"Tính các đạo hàm tại x = {x_prime}")
    
    current_coeffs = list(coeffs_desc)
    n = len(current_coeffs) - 1
    summary_data = []
    k_factorial = 1.0

    # Lặp qua từng cấp đạo hàm, từ k = 0 đến n
    for k in range(n + 1):
        print("\n" + "="*50 + f"\n--- BƯỚC {k+1}: TÍNH TOÁN CHO ĐẠO HÀM CẤP {k} ---")
        
        m = len(current_coeffs)
        deg = m - 1
        
        # --- Hiển thị bảng Horner cho bước hiện tại ---
        header = [f'a_{deg-i}' for i in range(deg + 1)]
        row_coeffs = [round(c, 10) for c in current_coeffs]
        row_products = ['']
        row_results = []

        b = current_coeffs[0]
        row_results.append(round(b, 10))
        
        quotient_coeffs = [b] # Hệ số của đa thức thương Q_{k+1}(x)
        
        for i in range(1, m):
            product = b * x_prime
            row_products.append(round(product, 10))
            b = product + current_coeffs[i]
            row_results.append(round(b, 10))
            if i < m - 1: # Hệ số cuối cùng là số dư
                quotient_coeffs.append(b)
        
        remainder = b # Số dư (R_{k+1})
        
        df = pd.DataFrame(
            [row_coeffs, row_products, row_results],
            index=['Hệ số vào', f'Nhân với x={x_prime}', 'Kết quả'],
            columns=header
        )
        display(df)
        
        # --- Tính giá trị đạo hàm P^(k)(x') = k! * R_{k+1} ---
        if k > 0:
            k_factorial *= k
        
        derivative_value = remainder * k_factorial
        summary_data.append([f"P^({k})(x')", derivative_value])
        
        print(f"-> Số dư R_{k+1} = {remainder:g}")
        print(f"-> Giá trị đạo hàm P^({k})(x') = R_{k+1} * {k}! = {remainder:g} * {k_factorial} = {derivative_value:g}")

        # Cập nhật hệ số cho vòng lặp tiếp theo
        current_coeffs = quotient_coeffs

    # --- In bảng tổng kết cuối cùng ---
    column_name_value = f"Giá trị tại x = {x_prime}"
    summary_df = pd.DataFrame(summary_data, columns=["Đạo hàm", column_name_value])
    print("\n" + "="*50 + f"\n--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = {x_prime} ---")
    
    # Áp dụng định dạng "{:g}" CHỈ cho cột chứa số
    display(summary_df.style.format("{:g}", subset=[column_name_value]))
    
    return summary_df

# --- 3. Thực thi thuật toán ---
try:
    # Gọi hàm tính toán, sử dụng các hệ số đã tính được ở ô trước
    ket_qua_dao_ham = tinh_P_va_cac_dao_ham(final_coeffs_descending[::-1], x_prime_input)

except NameError:
    print("LỖI: Biến 'final_coeffs_ascending' chưa được định nghĩa.")
    print("Vui lòng chạy lại ô code thực hiện phép nhân ma trận trước.")


Đa thức P(x) có hệ số (bậc giảm dần): [0.13508096841442727, -1.3152557319236655, 4.654216770888706, -7.251582892427784, 5.910993145757242, -4.261656349215418, 2.21393636363877]
Tính các đạo hàm tại x = 1.5

--- BƯỚC 1: TÍNH TOÁN CHO ĐẠO HÀM CẤP 0 ---


,a_6,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,0.135081,-1.315256,4.654217,-7.251583,5.910993,-4.261656,2.213936
Nhân với x=1.5,,0.202621,-1.668951,4.477898,-4.160527,2.625699,-2.453936
Kết quả,0.135081,-1.112634,2.985265,-2.773685,1.750466,-1.635958,-0.240000


-> Số dư R_1 = -0.24
-> Giá trị đạo hàm P^(0)(x') = R_1 * 0! = -0.24 * 1.0 = -0.24

--- BƯỚC 2: TÍNH TOÁN CHO ĐẠO HÀM CẤP 1 ---


,a_5,a_4,a_3,a_2,a_1,a_0
Hệ số vào,0.135081,-1.112634,2.985265,-2.773685,1.750466,-1.635958
Nhân với x=1.5,,0.202621,-1.365019,2.430369,-0.514974,1.853238
Kết quả,0.135081,-0.910013,1.620246,-0.343316,1.235492,0.217281


-> Số dư R_2 = 0.217281
-> Giá trị đạo hàm P^(1)(x') = R_2 * 1! = 0.217281 * 1.0 = 0.217281

--- BƯỚC 3: TÍNH TOÁN CHO ĐẠO HÀM CẤP 2 ---


...

,a_1,a_0
Hệ số vào,0.135081,-0.302148
Nhân với x=1.5,,0.202621
Kết quả,0.135081,-0.099527


-> Số dư R_6 = -0.099527
-> Giá trị đạo hàm P^(5)(x') = R_6 * 5! = -0.099527 * 120.0 = -11.9432

--- BƯỚC 7: TÍNH TOÁN CHO ĐẠO HÀM CẤP 6 ---


,a_0
Hệ số vào,0.135081
Nhân với x=1.5,
Kết quả,0.135081


-> Số dư R_7 = 0.135081
-> Giá trị đạo hàm P^(6)(x') = R_7 * 6! = 0.135081 * 720.0 = 97.2583

--- BẢNG TỔNG HỢP KẾT QUẢ CHO x = 1.5 ---


,Đạo hàm,Giá trị tại x = 1.5
0,P^(0)(x'),-0.24
1,P^(1)(x'),0.217281
2,P^(2)(x'),3.95725
3,P^(3)(x'),1.19057
4,P^(4)(x'),-15.6292
5,P^(5)(x'),-11.9432
6,P^(6)(x'),97.2583
